In [ ]:
import sys
from pathlib import Path
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.data_loader import (
    load_raw_data,
    save_processed_data,
    load_splits,
)

(20000, 24)